In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

In [3]:
# ann模型
def ann_model_search(train_X,train_y,valid_X,valid_y,output_shape=1,cv=5):
    def build_model(hidden_layers=1,layer_size=30,learning_rate=1e-3):
        model=Sequential()
        model.add(Dense(layer_size,activation='relu',input_shape=train_X.shape[1:]))
        for _ in range(hidden_layers):
            model.add(Dense(layer_size,activation='relu'))
        model.add(Dense(output_shape))
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss='mse',optimizer=optimizer)
        return model
    callbacks=[EarlyStopping(patience=3,min_delta=1e-3)]
    sklearn_model=keras.wrappers.scikit_learn.KerasRegressor(build_model)
    from scipy.stats import reciprocal
    param_distribution={
        'hidden_layers':[1,2,3],
        'layer_size':np.arange(10,100),
        'learning_rate':reciprocal(1e-4,1e-2),
    }
    from sklearn.model_selection import RandomizedSearchCV
    random_search_cv=RandomizedSearchCV(sklearn_model,param_distribution,n_iter=10,n_jobs=1,scoring='r2',cv=cv)
    random_search_cv.fit(train_X,train_y,epochs=30,callbacks=callbacks,validation_data=(valid_X,valid_y))
    return random_search_cv
# lstm模型
def lstm_model_search(train_X,train_y,valid_X,valid_y,output_shape=1,cv=3):
    def build_model(hidden_layers=1,layer_size=30,learning_rate=1e-3):
        model=Sequential()
        model.add(Dense(layer_size,activation='relu',input_shape=train_X.shape[1:]))
        for _ in range(hidden_layers-1):
            model.add(Dense(layer_size,activation='relu'))
        model.add(LSTM(layer_size,return_sequences=False))
        model.add(Dense(output_shape))
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss='mse',optimizer=optimizer)
        return model
    callbacks=[EarlyStopping(patience=5,min_delta=1e-3)]
    sklearn_model=keras.wrappers.scikit_learn.KerasRegressor(build_model)
    from scipy.stats import reciprocal
    param_distribution={
        'hidden_layers':[1,2,3],
        'layer_size':np.arange(10,100),
        'learning_rate':reciprocal(1e-4,1e-2),
    }
    from sklearn.model_selection import RandomizedSearchCV
    random_search_cv=RandomizedSearchCV(sklearn_model,param_distribution,n_iter=10,n_jobs=1,scoring='r2',cv=cv)
    random_search_cv.fit(train_X,train_y,epochs=30,callbacks=callbacks,validation_data=(valid_X,valid_y))
    return random_search_cv
# svm模型
def svm_model_search(train_X,train_y,output_shape=1,cv=5):
    def build_model(kernel='rbf',degree=3,C=1,epsilon=1e-2):
        model=SVR(kernel=kernel,degree=degree, C=C,epsilon=epsilon)
        if output_shape>1:
            model = MultiOutputRegressor(model)
        return model
    from scipy.stats import reciprocal
    if output_shape>1:
        param_distribution={
            'estimator__kernel':['rbf','poly','sigmoid','linear'],
            'estimator__degree':np.arange(2,5),
            'estimator__epsilon':reciprocal(1e-4,1e-2),
            'estimator__C':[1,2]
        }
    else:
        param_distribution={
            'kernel':['rbf','poly','sigmoid','linear'],
            'degree':np.arange(2,5),
            'epsilon':reciprocal(1e-4,1e-2),
            'C':[1,2]
        }
    model=build_model()
    from sklearn.model_selection import RandomizedSearchCV
    random_search_cv=RandomizedSearchCV(model,param_distribution,n_jobs=1,n_iter=5,scoring='r2',cv=cv)
    random_search_cv.fit(train_X,train_y)
    return random_search_cv


In [4]:
all_data=pd.read_csv('all_data.csv')
all_data

,时间,汕尾风速,风速1,风速2,风速3,风速4,汕尾风向,风向1,风向2,风向3,...,波浪周期,周期1,周期2,周期3,周期4,横向风速,纵向风速,前一小时潮水位,该时刻潮水位,后一小时潮水位
0,1993/1/1 0:00,3.784251,3.784251,3.784251,3.784251,3.784251,42.038391,42.025041,36.218454,45.892436,...,5.765194,5.574436,5.260206,6.023517,5.928466,2.534042,2.810549,710,1000,1130
1,1993/1/1 3:00,4.254679,4.254679,4.254679,4.254679,4.254679,43.295475,45.551842,45.919904,42.459024,...,5.791102,5.585522,5.241124,6.073677,5.966632,2.917692,3.096670,1190,1160,1230
2,1993/1/1 6:00,3.092321,3.092321,3.092321,3.092321,3.092321,53.562980,58.615288,64.004372,48.644659,...,5.813175,5.604241,5.247484,6.104573,5.991712,2.487804,1.836649,1410,1540,1580
3,1993/1/1 9:00,2.517260,2.517260,2.517260,2.517260,2.517260,52.355261,57.522090,60.417143,49.073149,...,5.846617,5.652039,5.300007,6.131289,6.012431,1.993199,1.537451,1640,1640,1500
4,1993/1/1 12:00,2.490680,2.490680,2.490680,2.490680,2.490680,34.608550,41.332865,40.942829,33.103663,...,5.836728,5.657309,5.319817,6.107663,5.989531,1.414623,2.049958,1370,1300,1330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1995/12/31 9:00,3.822870,3.822870,3.822870,3.822870,3.822870,36.904518,35.224151,33.103671,38.229080,...,5.754997,5.595432,5.200226,5.200226,5.200226,2.295569,3.056909,1640,1700,1720
8756,1995/12/31 12:00,3.687909,3.687909,3.687909,3.687909,3.687909,39.005398,38.382897,36.531597,40.113341,...,5.730869,5.618517,5.283145,5.283145,5.283145,2.321146,2.865825,1700,1600,1350
8757,1995/12/31 15:00,3.934920,3.934920,3.934920,3.934920,3.934920,39.072657,38.707012,36.570051,40.333080,...,5.640358,5.571240,5.300690,5.300690,5.300690,2.480201,3.054864,1100,950,870
8758,1995/12/31 18:00,4.243504,4.243504,4.243504,4.243504,4.243504,38.253099,38.602636,35.789978,39.855148,...,5.501376,5.437350,5.185452,5.185452,5.185452,2.627308,3.332354,880,930,960


In [5]:
MIN=all_data['横向风速'].min()
MAX=all_data['横向风速'].max()
all_data['横向风速scaled']=all_data['横向风速'].apply(lambda x:2*(x-MIN)/(MAX-MIN)-1)
MIN=all_data['纵向风速'].min()
MAX=all_data['纵向风速'].max()
all_data['纵向风速scaled']=all_data['纵向风速'].apply(lambda x:2*(x-MIN)/(MAX-MIN)-1)
all_data

,时间,汕尾风速,风速1,风速2,风速3,风速4,汕尾风向,风向1,风向2,风向3,...,周期2,周期3,周期4,横向风速,纵向风速,前一小时潮水位,该时刻潮水位,后一小时潮水位,横向风速scaled,纵向风速scaled
0,1993/1/1 0:00,3.784251,3.784251,3.784251,3.784251,3.784251,42.038391,42.025041,36.218454,45.892436,...,5.260206,6.023517,5.928466,2.534042,2.810549,710,1000,1130,0.158831,0.231581
1,1993/1/1 3:00,4.254679,4.254679,4.254679,4.254679,4.254679,43.295475,45.551842,45.919904,42.459024,...,5.241124,6.073677,5.966632,2.917692,3.096670,1190,1160,1230,0.198626,0.262188
2,1993/1/1 6:00,3.092321,3.092321,3.092321,3.092321,3.092321,53.562980,58.615288,64.004372,48.644659,...,5.247484,6.104573,5.991712,2.487804,1.836649,1410,1540,1580,0.154035,0.127402
3,1993/1/1 9:00,2.517260,2.517260,2.517260,2.517260,2.517260,52.355261,57.522090,60.417143,49.073149,...,5.300007,6.131289,6.012431,1.993199,1.537451,1640,1640,1500,0.102732,0.095396
4,1993/1/1 12:00,2.490680,2.490680,2.490680,2.490680,2.490680,34.608550,41.332865,40.942829,33.103663,...,5.319817,6.107663,5.989531,1.414623,2.049958,1370,1300,1330,0.042719,0.150220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1995/12/31 9:00,3.822870,3.822870,3.822870,3.822870,3.822870,36.904518,35.224151,33.103671,38.229080,...,5.200226,5.200226,5.200226,2.295569,3.056909,1640,1700,1720,0.134096,0.257935
8756,1995/12/31 12:00,3.687909,3.687909,3.687909,3.687909,3.687909,39.005398,38.382897,36.531597,40.113341,...,5.283145,5.283145,5.283145,2.321146,2.865825,1700,1600,1350,0.136749,0.237494
8757,1995/12/31 15:00,3.934920,3.934920,3.934920,3.934920,3.934920,39.072657,38.707012,36.570051,40.333080,...,5.300690,5.300690,5.300690,2.480201,3.054864,1100,950,870,0.153247,0.257716
8758,1995/12/31 18:00,4.243504,4.243504,4.243504,4.243504,4.243504,38.253099,38.602636,35.789978,39.855148,...,5.185452,5.185452,5.185452,2.627308,3.332354,880,930,960,0.168505,0.287399


In [6]:
all_data['前一小时潮水位除以1000']=all_data['前一小时潮水位'].apply(lambda x:x/1000)
all_data['该时刻潮水位除以1000']=all_data['该时刻潮水位'].apply(lambda x:x/1000)
all_data['后一小时潮水位除以1000']=all_data['后一小时潮水位'].apply(lambda x:x/1000)
all_data

,时间,汕尾风速,风速1,风速2,风速3,风速4,汕尾风向,风向1,风向2,风向3,...,横向风速,纵向风速,前一小时潮水位,该时刻潮水位,后一小时潮水位,横向风速scaled,纵向风速scaled,前一小时潮水位除以1000,该时刻潮水位除以1000,后一小时潮水位除以1000
0,1993/1/1 0:00,3.784251,3.784251,3.784251,3.784251,3.784251,42.038391,42.025041,36.218454,45.892436,...,2.534042,2.810549,710,1000,1130,0.158831,0.231581,0.71,1.00,1.13
1,1993/1/1 3:00,4.254679,4.254679,4.254679,4.254679,4.254679,43.295475,45.551842,45.919904,42.459024,...,2.917692,3.096670,1190,1160,1230,0.198626,0.262188,1.19,1.16,1.23
2,1993/1/1 6:00,3.092321,3.092321,3.092321,3.092321,3.092321,53.562980,58.615288,64.004372,48.644659,...,2.487804,1.836649,1410,1540,1580,0.154035,0.127402,1.41,1.54,1.58
3,1993/1/1 9:00,2.517260,2.517260,2.517260,2.517260,2.517260,52.355261,57.522090,60.417143,49.073149,...,1.993199,1.537451,1640,1640,1500,0.102732,0.095396,1.64,1.64,1.50
4,1993/1/1 12:00,2.490680,2.490680,2.490680,2.490680,2.490680,34.608550,41.332865,40.942829,33.103663,...,1.414623,2.049958,1370,1300,1330,0.042719,0.150220,1.37,1.30,1.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1995/12/31 9:00,3.822870,3.822870,3.822870,3.822870,3.822870,36.904518,35.224151,33.103671,38.229080,...,2.295569,3.056909,1640,1700,1720,0.134096,0.257935,1.64,1.70,1.72
8756,1995/12/31 12:00,3.687909,3.687909,3.687909,3.687909,3.687909,39.005398,38.382897,36.531597,40.113341,...,2.321146,2.865825,1700,1600,1350,0.136749,0.237494,1.70,1.60,1.35
8757,1995/12/31 15:00,3.934920,3.934920,3.934920,3.934920,3.934920,39.072657,38.707012,36.570051,40.333080,...,2.480201,3.054864,1100,950,870,0.153247,0.257716,1.10,0.95,0.87
8758,1995/12/31 18:00,4.243504,4.243504,4.243504,4.243504,4.243504,38.253099,38.602636,35.789978,39.855148,...,2.627308,3.332354,880,930,960,0.168505,0.287399,0.88,0.93,0.96


In [7]:
data=all_data[['横向风速scaled','纵向风速scaled','前一小时潮水位除以1000','该时刻潮水位除以1000','后一小时潮水位除以1000','汕尾波高']]
data

,横向风速scaled,纵向风速scaled,前一小时潮水位除以1000,该时刻潮水位除以1000,后一小时潮水位除以1000,汕尾波高
0,0.158831,0.231581,0.71,1.00,1.13,1.148035
1,0.198626,0.262188,1.19,1.16,1.23,1.044409
2,0.154035,0.127402,1.41,1.54,1.58,0.954029
3,0.102732,0.095396,1.64,1.64,1.50,0.875360
4,0.042719,0.150220,1.37,1.30,1.33,0.813603
...,...,...,...,...,...,...
8755,0.134096,0.257935,1.64,1.70,1.72,1.059991
8756,0.136749,0.237494,1.70,1.60,1.35,0.997827
8757,0.153247,0.257716,1.10,0.95,0.87,0.963268
8758,0.168505,0.287399,0.88,0.93,0.96,0.953600


In [8]:
def create_data(data,input_points=8,pred_points=1):
    X_data=[]
    Y_data=[]
    for i in range(8760-input_points-pred_points+1):
        X_data.append(data.iloc[i:i+input_points,[0,1,2,3,4,5]].values)
        Y_data.append(data.iloc[i+input_points:i+input_points+pred_points,5].values)
    X_data=np.array(X_data)
    X_data=X_data.reshape(X_data.shape[0],X_data.shape[1]*X_data.shape[2])
    return X_data,np.array(Y_data)
def create_lstm_data(data,input_points=8,pred_points=1):
    X_data=[]
    Y_data=[]
    for i in range(8760-input_points-pred_points+1):
        X_data.append(data.iloc[i:i+input_points,[0,1,2,3,4,5]].values)
        Y_data.append(data.iloc[i+input_points:i+input_points+pred_points,5].values)
    X_data=np.array(X_data)
    return X_data,np.array(Y_data)
X,y=create_data(data,24,8)
print(X.shape)
print(y.shape)

(8729, 144)
(8729, 8)


In [9]:
train_X=X[0:6983]
test_X=X[6983:]
train_y=y[0:6983]
test_y=y[6983:]

In [10]:
svm_search=svm_model_search(train_X,train_y,output_shape=8,cv=5)

/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/De

/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/De

/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/De

/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/De

/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/De

/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/De

/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/Desktop/DL_Learn/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/mahinmy/De

In [11]:
print(svm_search.best_params_)
print(svm_search.best_score_)
print(svm_search.best_estimator_)

{'estimator__C': 2, 'estimator__degree': 3, 'estimator__epsilon': 0.0028297441385735735, 'estimator__kernel': 'linear'}
0.8425779130195218
MultiOutputRegressor(estimator=SVR(C=2, cache_size=200, coef0=0.0, degree=3,
                                   epsilon=0.0028297441385735735,
                                   gamma='auto_deprecated', kernel='linear',
                                   max_iter=-1, shrinking=True, tol=0.001,
                                   verbose=False),
                     n_jobs=None)


In [12]:
model=svm_search.best_estimator_
y_pred=model.predict(test_X)
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print(mean_absolute_error(y_pred,test_y))
print(mean_squared_error(y_pred,test_y))
print(r2_score(y_pred,test_y))

0.11416752777459709
0.03902913066063096
0.8349698658789
